# PageRank - Explicado

## Definiciones  

 - $PR(usuario_i)$: PageRank calculado para la el usuario $i$.
 - Backlink: Si la el usuario $A$ sigue a $B$, entonces se dice que el usuario $B$ tiene un *backlink* desde $A$.
 
## ¿Qué es PageRank?  

En resumen, PageRank es una especie de "voto", otorgado por todos los usuarios de twitter qué tan importante un determinado usuario es. Cada follow cuenta como un "voto".

## ¿Cómo funciona?  

Supongamos que tenemos $N$ usuarios en nuestro universo. Y si el usuario $A$ es seguido por $u_1$, $u_2$, $...$, $u_n$, el PageRank del usuario $A$ está determinado por:  

$$PR(A) = \frac{1 - d}{N} + d \left( \frac{PR(u_1)}{L(u_1)} +  \frac{PR(u_2)}{L(u_2)} +\space...\space+  \frac{PR(u_n)}{L(u_n)} \right)$$

Nuevos términos: 

 -  $L(u_n)$ - El término $L(u_n)$ se refiere el número de usuarios que el usuario $u_n$ sigue, su contribución al PageRank de cualquiera de las personas que sigue se ve "diluido" entre más usuarios siga.
 - $d$ - es conocida como factor de moderación. Un valor común es $0.85$.  

## ¿Iterativo?  

No es suficiente con calcular el PageRank una sola vez, puesto que los valores fluctuan, la importancia de un determinado usuario no puede ser calculado en una sola operación... sin embargo, con el tiempo, los valores de PageRank convergen, es decir, se mantienen estables. La fórmula en realidad se divide en:  

Cuando $t = 0$:  

$$PR(u_i;0) = \frac{1}{N}$$

Para cualquier otro valor de $t$:  

$$PR(u_i;t) = \frac{1 - d}{N} + d \sum_{u_j \in S(u_i) } \frac{PR(u_j;t-1)}{L(u_j)}$$

In [ ]:
import json
from collections import defaultdict
from copy import deepcopy

In [ ]:
def invert_graph(graph: dict):

    followers_graph = defaultdict(set)

    for user, followed_users in graph.items():
        for followed_user in followed_users:
            followers_graph[followed_user].add(user)
    
    return dict(followers_graph)

In [ ]:
def calculate_pr(graph, d = 0.85, iterations = 100, debug = False):
    inverted_graph = invert_graph(graph)
    
    N = len(graph)
    S = lambda u: inverted_graph.get(u, set())
    L = lambda u: len(graph[u])

    initial_pr = { user: 1 / N for user in graph }
    previous_pr = deepcopy(initial_pr)

    for it in range(1, iterations + 1):
        iteration_pr = {}
        for u_i in graph:
            simple_page_rank = (1 - d) / N
            contributions = 0
            for u_j in S(u_i):
                contributions += previous_pr[u_j] / L(u_j)
            iteration_pr[u_i] = simple_page_rank + d * contributions
        previous_pr = deepcopy(iteration_pr)

    return previous_pr

## Un micro ejemplo  

In [ ]:
twitter_graph = {}
with open("twitter_closed.jsonl") as readable:
    for line in readable:
        user = json.loads(line)
        twitter_graph[user["user_id"]] = user["following_id"]

In [ ]:
results = calculate_pr(twitter_graph, iterations = 10)

In [ ]:
results[2582486138]

In [ ]:
results[1306760288243462147]